In [1]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    DataCollatorForTokenClassification,
    Trainer,
    TrainingArguments,
    AutoModelForTokenClassification,
    pipeline
)
from rich import print
import evaluate
import numpy as np

In [2]:
wnut = load_dataset("wnut_17")
print(wnut["train"][0])

{
    'id': '0',
    'tokens': [
        '@paulwalk',
        'It',
        "'s",
        'the',
        'view',
        'from',
        'where',
        'I',
        "'m",
        'living',
        'for',
        'two',
        'weeks',
        '.',
        'Empire',
        'State',
        'Building',
        '=',
        'ESB',
        '.',
        'Pretty',
        'bad',
        'storm',
        'here',
        'last',
        'evening',
        '.'
    ],
    'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0]
}

In [3]:
label_list = wnut["train"].features[f"ner_tags"].feature.names
print(label_list)

[
    'O',
    'B-corporation',
    'I-corporation',
    'B-creative-work',
    'I-creative-work',
    'B-group',
    'I-group',
    'B-location',
    'I-location',
    'B-person',
    'I-person',
    'B-product',
    'I-product'
]

In [4]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [5]:
example = wnut["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

[
    '[CLS]',
    '@',
    'paul',
    '##walk',
    'it',
    "'",
    's',
    'the',
    'view',
    'from',
    'where',
    'i',
    "'",
    'm',
    'living',
    'for',
    'two',
    'weeks',
    '.',
    'empire',
    'state',
    'building',
    '=',
    'es',
    '##b',
    '.',
    'pretty',
    'bad',
    'storm',
    'here',
    'last',
    'evening',
    '.',
    '[SEP]'
]

In [6]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [7]:
tokenized_wnut = wnut['train'].map(tokenize_and_align_labels, batched=True)

In [8]:
print(tokenized_wnut[0])

{
    'id': '0',
    'tokens': [
        '@paulwalk',
        'It',
        "'s",
        'the',
        'view',
        'from',
        'where',
        'I',
        "'m",
        'living',
        'for',
        'two',
        'weeks',
        '.',
        'Empire',
        'State',
        'Building',
        '=',
        'ESB',
        '.',
        'Pretty',
        'bad',
        'storm',
        'here',
        'last',
        'evening',
        '.'
    ],
    'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0],
    'input_ids': [
        101,
        1030,
        2703,
        17122,
        2009,
        1005,
        1055,
        1996,
        3193,
        2013,
        2073,
        1045,
        1005,
        1049,
        2542,
        2005,
        2048,
        3134,
        1012,
        3400,
        2110,
        2311,
        1027,
        9686,
        2497,
        1012,
        3492,
        2919,
        4040,
        2182,
        2197,
        3944,
        1012,
        102
    ],
    'attention_mask': [
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1
    ],
    'labels': [
        -100,
        0,
        -100,
        -100,
        0,
        0,
        -100,
        0,
        0,
        0,
        0,
        0,
        0,
        -100,
        0,
        0,
        0,
        0,
        0,
        7,
        8,
        8,
        0,
        7,
        -100,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        -100
    ]
}

In [8]:
seqeval = evaluate.load("seqeval")
labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [9]:
id2label = {
    0: "O",
    1: "B-corporation",
    2: "I-corporation",
    3: "B-creative-work",
    4: "I-creative-work",
    5: "B-group",
    6: "I-group",
    7: "B-location",
    8: "I-location",
    9: "B-person",
    10: "I-person",
    11: "B-product",
    12: "I-product",
}
label2id = {
    "O": 0,
    "B-corporation": 1,
    "I-corporation": 2,
    "B-creative-work": 3,
    "I-creative-work": 4,
    "B-group": 5,
    "I-group": 6,
    "B-location": 7,
    "I-location": 8,
    "B-person": 9,
    "I-person": 10,
    "B-product": 11,
    "I-product": 12,
}

In [13]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="pt")

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=13, id2label=id2label, label2id=label2id
)
log_step = 40
training_args = TrainingArguments(
    output_dir="my_awesome_wnut_model",
    warmup_ratio=0.1,
    lr_scheduler_type='cosine',
    weight_decay=0.001,
    learning_rate=1e-4,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=16,
    fp16=True,
    logging_strategy='steps',
    save_strategy='steps',
    eval_strategy='steps',
    save_steps=log_step,
    eval_steps=log_step,
    logging_steps=log_step,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    report_to="none",
    num_train_epochs=3,
    optim='adafactor',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
40,0.664500,0.254968,0.537010,0.342910,0.418552,0.941644
80,0.098000,0.266896,0.579861,0.309546,0.403625,0.943183


***** train metrics *****
  epoch                    =        3.0
  total_flos               =   151279GF
  train_loss               =     0.3776
  train_runtime            = 0:00:05.20
  train_samples_per_second =   1957.078
  train_steps_per_second   =     15.569


In [14]:
classifier = pipeline("ner", model="my_awesome_wnut_model")

text = "The Golden State Warriors are an American professional basketball team based in San Francisco."
classifier(text)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'entity': 'B-location',
  'score': np.float32(0.48355395),
  'index': 2,
  'word': 'golden',
  'start': 4,
  'end': 10},
 {'entity': 'I-location',
  'score': np.float32(0.47697803),
  'index': 3,
  'word': 'state',
  'start': 11,
  'end': 16},
 {'entity': 'B-location',
  'score': np.float32(0.60752493),
  'index': 13,
  'word': 'san',
  'start': 80,
  'end': 83},
 {'entity': 'B-location',
  'score': np.float32(0.43599966),
  'index': 14,
  'word': 'francisco',
  'start': 84,
  'end': 93}]